In [1]:
!pip install transformers[torch] datasets evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# Performs supervised fine-tuning LazBF-ESM and LazDEF-ESM

# Imports
import torch
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.utils import resample
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from scipy.stats import spearmanr
from sklearn.metrics import ndcg_score
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, normalize
from sklearn.model_selection import KFold
from sklearn.utils import resample
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as ticker
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import ndcg_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, SVC
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling
from datasets import Dataset
from evaluate import load
from datasets import load_metric
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load sequences from csv
df = pd.read_csv('./drive/MyDrive/Data/LazBF_sequences.csv')
LazBF_sequences = df['sequences'].tolist()
LazBF_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/Data/LazBF_sample.csv')
LazBF_sample = df['sequences'].tolist()
LazBF_sample_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/Data/LazDEF_sequences.csv')
LazDEF_sequences = df['sequences'].tolist()
LazDEF_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/Data/LazDEF_sample.csv')
LazDEF_sample = df['sequences'].tolist()
LazDEF_sample_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/Data/LazBCDEF_sequences.csv')
LazBCDEF_sequences = df['sequences'].tolist()
LazBCDEF_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/Data/LazBCDEF_sample.csv')
LazBCDEF_sample = df['sequences'].tolist()
LazBCDEF_sample_labels = df['labels'].tolist()

LazBF_ft = AutoModelForSequenceClassification.from_pretrained('facebook/esm2_t33_650M_UR50D').to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")

# Supervised fine-tune LazBF-ESM
train = Dataset.from_dict(tokenizer(LazBF_sequences))
train = train.add_column("labels", LazBF_labels)
train = train.shuffle(seed=16)

test = Dataset.from_dict(tokenizer(LazBF_sample))
test = test.add_column("labels", LazBF_sample_labels)

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/Models/LazBF_ft",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    fp16=True,
    load_best_model_at_end=True,
    gradient_accumulation_steps=2,
)

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=LazBF_ft,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics
)
trainer.train()

In [2]:
LazDEF_ft = AutoModelForSequenceClassification.from_pretrained('facebook/esm2_t33_650M_UR50D').to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")

# Supervised fine-tune LazDEF-ESM
train = Dataset.from_dict(tokenizer(LazDEF_sequences))
train = train.add_column("labels", LazDEF_labels)
train = train.shuffle(seed=42)

test = Dataset.from_dict(tokenizer(LazDEF_sample))
test = test.add_column("labels", LazDEF_sample_labels)

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/Models/LazDEF_ft",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    fp16=True,
    load_best_model_at_end=True,
    gradient_accumulation_steps=2,
)

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=LazDEF_ft,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strat

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
0,0.031600,0.026966,0.991960


TrainOutput(global_step=2441, training_loss=0.04525823839274355, metrics={'train_runtime': 4424.5114, 'train_samples_per_second': 282.517, 'train_steps_per_second': 0.552, 'total_flos': 7.322542384909824e+16, 'train_loss': 0.04525823839274355, 'epoch': 0.999795207864018})

In [4]:
LazBCDEF_ft = AutoModelForSequenceClassification.from_pretrained('facebook/esm2_t33_650M_UR50D').to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")

# Supervised fine-tune LazDEF-ESM
train = Dataset.from_dict(tokenizer(LazBCDEF_sequences, padding='longest'))
train = train.add_column("labels", LazBCDEF_labels)
train = train.shuffle(seed=42)

test = Dataset.from_dict(tokenizer(LazBCDEF_sample, padding='longest'))
test = test.add_column("labels", LazBCDEF_sample_labels)

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/Models/LazBCDEF_ft",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    fp16=True,
    load_best_model_at_end=True,
    gradient_accumulation_steps=2,
)

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=LazBCDEF_ft,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.121600,0.120519,0.957620


TrainOutput(global_step=2441, training_loss=0.13213378081112698, metrics={'train_runtime': 3894.0005, 'train_samples_per_second': 321.007, 'train_steps_per_second': 0.627, 'total_flos': 6.346203400255181e+16, 'train_loss': 0.13213378081112698, 'epoch': 0.999795207864018})